# Preços de Imóveis em São Paulo
Neste notebook irei treinar um modelo para fazer a previsão do proço de venda para imóveis em São Paulo, entretanto o objetivo final é fazer o deploy de um modelo para fazer a alimenação de uma aplicação web.

Como o objetivo é fazer é focar na construção de um webapp  e os passos para subir uma aplicação, a etapa de análise exploratória de dados foi propositalmente suprimida.

Adiante iremos ver como exportar e importar o modelo usando a biblioteca 'joblib'.

## Dados dos imóveis

Os dados que serão utilizados no desenvolvimento foram obtidos nesse [link](https://www.kaggle.com/argonalyst/sao-paulo-real-estate-sale-rent-april-2019) e disponibilizados publicamente pela startup OpenImob.

Para caminharmos direto ao objeto desse notebook irei utilizar os dados já [limpos](https://www.dropbox.com/s/h8blgaphkfpqsn5/sao-paulo-properties-april-2019.csv?dl=1) pelo [Carlos Melo](http://www.sigmoidal.ai).



In [1]:
#importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#importar o dataset
#o download do arquivo foi feito previamente pelo link apontado.
file = "files\sao-paulo-properties-april-2019.csv"
df = pd.read_csv(file)

In [3]:
df.head()


,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


Repare que há informações desnecessária, como por exemplo na variável 'District' há o nome do bairro e depois a cidade *São Paulo*.

In [4]:
#removendo a informação desnecessária
df_clean = df.copy()

#limpando o nome da cidade
df_clean['District'] = df_clean['District'].apply(lambda x: x.split('/')[0])

df_clean.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim,rent,apartment,-23.525025,-46.482436


# Criando um modelo Ensemble de Machine Learning

No próximo passo iremos instalar as dependências necessárias e criar um modelo de Machine Learning, mas por que usar apenas um modelo de machine learning, por que apenas uma forma de fazer previsão? Podemos utilizar o melhor de cada modelo e até combiná-los entre si.

Caso tenha interesse em conhecer mais sobre o modelo Ensemble, pode começar pelo [post](https://mabittar.github.io/EnsembleMethod/) que eu publiquei.

In [5]:
!pip install xgboost sklearn

In [12]:
# 1. importando as bibliotecas necessárias
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [7]:
#variáveis dummies
df_clean = pd.get_dummies(df_clean)

# 2.separando variáveis de x e y
X = df_clean.drop('Price', axis=1)
y = df_clean['Price']

# 2.1 dividindo o dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(9548, 111) (4092, 111) (9548,) (4092,)


In [9]:
# utilizando todos núcleos do processador
import multiprocessing
n_jobs = multiprocessing.cpu_count()-1

In [17]:
# 3. instanciando os modelo
model = RandomForestRegressor(n_jobs = n_jobs, random_state=42)
model.fit(X_train, y_train)

# fazer as previsões em cima do dataset de teste
y_pred = model.predict(X_test)

# métricas de avaliação
print("R2: \t{:.4f}".format(r2_score(y_test, y_pred)))
print("MAE: \t{:.4f}".format(mean_absolute_error(y_test, y_pred)))
print("MSE: \t{:.4f}".format(mean_squared_error(y_test, y_pred)))


r2: 	0.9215
MAE: 	46517.8792
MSE: 	28610978800.6684


## Salvando o Modelo

Agora que o nosso modelo está treinado e é capaz de realizar previsões iremos utilizar a biblioteca joblib para exportá-lo.

In [18]:
from joblib import dump, load

dump(model, 'model.joblib') 

['model.joblib']

Uma vez que criamos nosso arquivo para exportar o modelo devemos também exportar as features (variáveis) que utilizamos para treiná-lo.

In [20]:
# salvando os nomes das features
features = X_train.columns.values

dump (features, 'features.names')

['features.names']

## Carregando o modelo

Agora que exportamos nosso modelo podemos também importá-lo, podemos ver nos próximas etapas.

Irei aproveitar também para exportar os dados necessários para criarmos o arquivo json que irá alimentar a API.

In [21]:
#importando o modelo
novo_modelo = load('model.joblib')

#verificando o tipo
type(novo_modelo)

sklearn.ensemble._forest.RandomForestRegressor

In [24]:
import json
#exportando os bairros
json_object = json.dumps(dict(zip(X.columns.values, np.zeros(X.shape[0]).astype(int).tolist())), indent = 4)



In [29]:
#Salvando o nome dos bairros em um arquivo txt
#como esse arquivo foi desenvolvido diretamente no VSCode instalado no windows pt-br não preciso realizar  etapa para instalar os carecters acentuados
bairros = [f.split("District_")[1] for f in df_clean.columns if "District_" in f]

dump(sorted(bairros), 'bairros.joblib')


['bairros.joblib']